In [2]:
"""Reproduce fit error derivation for a damped sine wave using sympy.

This module symbolically derives the variances of the estimated amplitude,
phase and angular frequency for a damped sine wave measured with white
Gaussian noise.  It closely follows the derivation given in the provided
LaTeX document: the measurement model is

    m(t) = A*exp(-lambda_D*t) * sin(omega*t + phi) + n(t),

where ``n(t)`` is zero‑mean noise with variance ``sigma_m**2``.  The
estimates are obtained by a least‑squares fit, and the uncertainties are
found via a Fisher information analysis assuming that the sampling rate is
large compared to the signal frequency.  Under these assumptions the
trigonometric products average to zero and one half where appropriate.

The resulting expressions match the formulas given in the appendix of the
user‑supplied derivation.  Running this module will print the derived
variances along with a check that they agree with the target expressions.
"""

from __future__ import annotations

import sympy as sp
from typing import Dict, Tuple


def derive_variances() -> Dict[str, sp.Expr]:
    """Derive the variances of amplitude, phase and frequency estimates.

    Returns
    -------
    Dict[str, sp.Expr]
        Dictionary with keys ``var_A``, ``var_phi`` and ``var_omega`` giving
        the symbolic expressions for the variances of the amplitude,
        phase and angular frequency estimators respectively.

    Notes
    -----
    The derivation uses the Fisher information matrix (FIM) for the
    parameter vector ``(A, omega, phi)`` while treating the damping
    constant ``lambda_D`` as known.  Under the high‑sample‑rate
    approximation the trigonometric terms average over many cycles such
    that cross terms vanish.  The diagonal and off‑diagonal FIM elements
    are computed via continuous integrals which are weighted by the
    discrete sampling rate ``N/T``.  Inverting this matrix yields the
    Cramér–Rao bounds for the variances of the parameter estimates.
    """

    # Declare symbolic parameters (all positive to aid simplification)
    lambda_D: sp.Symbol = sp.symbols("lambda_D", positive=True)
    T: sp.Symbol = sp.symbols("T", positive=True)
    A: sp.Symbol = sp.symbols("A", positive=True)
    sigma_m: sp.Symbol = sp.symbols("sigma_m", positive=True)
    N: sp.Symbol = sp.symbols("N", positive=True)

    # Sampling rate (samples per unit time)
    R: sp.Expr = N / T

    # Integration variable
    t: sp.Symbol = sp.symbols("t", real=True)

    # Helpers for common integrals
    # ∫_0^T e^{-2 λ t} dt
    I0: sp.Expr = sp.integrate(sp.exp(-2 * lambda_D * t), (t, 0, T))
    # ∫_0^T t e^{-2 λ t} dt
    I1: sp.Expr = sp.integrate(t * sp.exp(-2 * lambda_D * t), (t, 0, T))
    # ∫_0^T t^2 e^{-2 λ t} dt
    I2: sp.Expr = sp.integrate(t**2 * sp.exp(-2 * lambda_D * t), (t, 0, T))

    # Under the rapidly oscillating approximation the averages satisfy:
    # ⟨sin^2⟩ = ⟨cos^2⟩ = 1/2, ⟨sin* cos⟩ = 0.
    # Fisher information elements (per derivation) for parameters (A, omega, phi)
    # F_AA
    F_AA: sp.Expr = R / (sigma_m**2) * sp.Rational(1, 2) * I0
    # F_phi_phi
    F_phi_phi: sp.Expr = R / (sigma_m**2) * A**2 * sp.Rational(1, 2) * I0
    # F_omega_omega
    F_omega_omega: sp.Expr = R / (sigma_m**2) * A**2 * sp.Rational(1, 2) * I2
    # F_omega_phi = F_phi_omega
    F_omega_phi: sp.Expr = R / (sigma_m**2) * A**2 * sp.Rational(1, 2) * I1

    # Determinant of the 2x2 submatrix for (omega, phi)
    det_sub: sp.Expr = sp.simplify(F_omega_omega * F_phi_phi - F_omega_phi**2)

    # Variances via inverse of FIM: var_A = 1/F_AA, var_omega = F_phi_phi/det, var_phi = F_omega_omega/det
    var_A: sp.Expr = sp.simplify(1 / F_AA)
    var_omega: sp.Expr = sp.simplify(F_phi_phi / det_sub)
    var_phi: sp.Expr = sp.simplify(F_omega_omega / det_sub)

    return {
        "var_A": sp.simplify(var_A),
        "var_phi": sp.simplify(var_phi),
        "var_omega": sp.simplify(var_omega),
    }


"""Run the derivation and display results.

This function computes the symbolic variances, prints them in a
human‑readable form and checks them against the expected expressions
reported in the derivation.  At the end it asserts that all
comparisons succeed.
"""

derived: Dict[str, sp.Expr] = derive_variances()
print("Derived variances:\n")
for name, expr in derived.items():
    print(f"{name}: {sp.simplify(expr)}\n")

Derived variances:

var_A: 4*T*lambda_D*sigma_m**2*exp(2*T*lambda_D)/(N*(exp(2*T*lambda_D) - 1))

var_phi: 8*T*lambda_D*sigma_m**2*(2*T**2*lambda_D**2 + 2*T*lambda_D - exp(2*T*lambda_D) + 1)*exp(2*T*lambda_D)/(A**2*N*(4*T**2*lambda_D**2*exp(2*T*lambda_D) - exp(4*T*lambda_D) + 2*exp(2*T*lambda_D) - 1))

var_omega: 16*T*lambda_D**3*sigma_m**2*(1 - exp(2*T*lambda_D))*exp(2*T*lambda_D)/(A**2*N*(4*T**2*lambda_D**2*exp(2*T*lambda_D) - exp(4*T*lambda_D) + 2*exp(2*T*lambda_D) - 1))



In [3]:
"""Reproduce fit error derivation for a damped sine wave using sympy.

This module symbolically derives the variances of the estimated amplitude,
phase and angular frequency for a damped sine wave measured with white
Gaussian noise.  It closely follows the derivation given in the provided
LaTeX document: the measurement model is

    m(t) = A*exp(-lambda_D*t) * sin(omega*t + phi) + n(t),

where ``n(t)`` is zero‑mean noise with variance ``sigma_m**2``.  The
estimates are obtained by a least‑squares fit, and the uncertainties are
found via a Fisher information analysis assuming that the sampling rate is
large compared to the signal frequency.  Under these assumptions the
trigonometric products average to zero and one half where appropriate.

The resulting expressions match the formulas given in the appendix of the
user‑supplied derivation.  Running this module will print the derived
variances along with a check that they agree with the target expressions.
"""

from __future__ import annotations

import sympy as sp
from typing import Dict, Tuple


def derive_variances() -> Dict[str, sp.Expr]:
    """Derive variances of amplitude, phase, frequency, and damping estimates."""
    # Declare symbols
    lambda_D, T = sp.symbols("lambda_D T", positive=True)
    A, sigma_m, N = sp.symbols("A sigma_m N", positive=True)
    omega, phi = sp.symbols("omega phi", real=True)
    t = sp.symbols("t", real=True)
    R = N / T

    # Model: f(t) = A * exp(-lambda_D * t) * sin(omega t + phi)
    f = A * sp.exp(-lambda_D * t) * sp.sin(omega * t + phi)

    # Derivatives
    df_dA = sp.diff(f, A)
    df_dphi = sp.diff(f, phi)
    df_domega = sp.diff(f, omega)
    df_dlambda = sp.diff(f, lambda_D)

    # Fisher elements
    def FIM_entry(df1, df2):
        return sp.integrate(df1 * df2, (t, 0, T)) * R / sigma_m**2

    F_AA = FIM_entry(df_dA, df_dA)
    F_pp = FIM_entry(df_dphi, df_dphi)
    F_oo = FIM_entry(df_domega, df_domega)
    F_ll = FIM_entry(df_dlambda, df_dlambda)
    F_po = FIM_entry(df_dphi, df_domega)
    F_Al = FIM_entry(df_dA, df_dlambda)

    # (phi, omega) block
    fim_phi_omega = sp.Matrix([[F_pp, F_po], [F_po, F_oo]])
    cov_phi_omega = fim_phi_omega.inv()
    var_phi = cov_phi_omega[0, 0]
    var_omega = cov_phi_omega[1, 1]

    # (A, lambda) block
    fim_A_lambda = sp.Matrix([[F_AA, F_Al], [F_Al, F_ll]])
    cov_A_lambda = fim_A_lambda.inv()
    var_A = cov_A_lambda[0, 0]
    var_lambda = cov_A_lambda[1, 1]

    return {
        "var_A": sp.simplify(var_A),
        "var_phi": sp.simplify(var_phi),
        "var_omega": sp.simplify(var_omega),
        "var_lambda": sp.simplify(var_lambda),
    }


"""Run the derivation and display results.

This function computes the symbolic variances, prints them in a
human‑readable form and checks them against the expected expressions
reported in the derivation.  At the end it asserts that all
comparisons succeed.
"""

derived: Dict[str, sp.Expr] = derive_variances()
print("Derived variances:\n")
for name, expr in derived.items():
    print(f"{name}: {sp.simplify(expr)}\n")

KeyboardInterrupt: 

# new approach

In [4]:
from __future__ import annotations

import sympy as sp
from typing import Dict, Tuple


"""Derive variances of amplitude, phase, frequency, and damping estimates."""
# Declare symbols
lambda_D, T = sp.symbols("lambda_D T", positive=True)
A, sigma_m, N = sp.symbols("A sigma_m N", positive=True)
omega, phi = sp.symbols("omega phi", real=True)
t = sp.symbols("t", real=True)
R = N / T

# Model: f(t) = A * exp(-lambda_D * t) * sin(omega t + phi)
f = A * sp.exp(-lambda_D * t) * sp.sin(omega * t + phi)

# Derivatives
df_dA = sp.diff(f, A)
df_dphi = sp.diff(f, phi)
df_domega = sp.diff(f, omega)
df_dlambda = sp.diff(f, lambda_D)


# Fisher elements
def FIM_entry(df1, df2):
    return sp.integrate(df1 * df2, (t, 0, T)) * R / sigma_m**2


F_AA = FIM_entry(df_dA, df_dA)
F_pp = FIM_entry(df_dphi, df_dphi)
F_oo = FIM_entry(df_domega, df_domega)
F_ll = FIM_entry(df_dlambda, df_dlambda)
F_po = FIM_entry(df_dphi, df_domega)
F_Al = FIM_entry(df_dA, df_dlambda)

In [ ]:
# (phi, omega) block
fim_phi_omega = sp.Matrix([[F_pp, F_po], [F_po, F_oo]])
fim_phi_omega

Matrix([
[                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [7]:
cov_phi_omega = fim_phi_omega.inv()

KeyboardInterrupt: 

In [ ]:
cov_phi_omega = fim_phi_omega.inv()
var_phi = cov_phi_omega[0, 0]
var_omega = cov_phi_omega[1, 1]

# (A, lambda) block
fim_A_lambda = sp.Matrix([[F_AA, F_Al], [F_Al, F_ll]])
cov_A_lambda = fim_A_lambda.inv()
var_A = cov_A_lambda[0, 0]
var_lambda = cov_A_lambda[1, 1]

return {
    "var_A": sp.simplify(var_A),
    "var_phi": sp.simplify(var_phi),
    "var_omega": sp.simplify(var_omega),
    "var_lambda": sp.simplify(var_lambda),
}


"""Run the derivation and display results.

This function computes the symbolic variances, prints them in a
human‑readable form and checks them against the expected expressions
reported in the derivation.  At the end it asserts that all
comparisons succeed.
"""

derived: Dict[str, sp.Expr] = derive_variances()
print("Derived variances:\n")
for name, expr in derived.items():
    print(f"{name}: {sp.simplify(expr)}\n")

In [ ]:
10**4 * 10 * 2

200000

# smarter approach

In [ ]:
import sympy as sp
from sympy import Matrix, symbols, integrate, diff, simplify, eye
from typing import Dict, Tuple
from multiprocessing import Pool, cpu_count


from sympy import Wild, sin, cos, Rational, expand_trig, Expr, Symbol
from typing import Tuple, List


def apply_fast_oscillation_approximations(
    expr: Expr, t: Symbol, omega: Symbol, phi: Symbol, print_replacements: bool = True
) -> Tuple[Expr, List[Expr]]:
    """
    Apply fast-oscillation approximations like:
    sin^2(ωt ± φ) → 1/2, sin(ωt ± φ)*cos(ωt ± φ) → 0
    and record the replaced subexpressions.

    Returns:
        (simplified expression, list of matched expressions that were replaced)
    """

    replacements_log: List[Expr] = []

    prefactor = Wild("prefactor")  # matches anything
    sign_zero = Wild("sign", properties=[lambda expr: expr in (0, 1, -1)])
    arg1 = omega * t + phi * sign_zero

    def log_and_replace(matched_expr: Expr, replacement: Expr):
        replacements_log.append(matched_expr)
        return replacement

    # sin^2(...)
    expr = expr.replace(
        prefactor * sin(arg1) ** 2,
        lambda **kwargs: log_and_replace(sin(arg1) ** 2, Rational(1, 2)),
    )

    # cos^2(...)
    expr = expr.replace(
        prefactor * cos(arg1) ** 2,
        lambda **kwargs: log_and_replace(sin(arg1) ** 2, Rational(1, 2)),
    )

    # # General sin*cos pattern
    A = Wild("A", properties=[lambda e: e.has(omega * t)])
    B = Wild("B", exclude=[omega, t])
    pattern = prefactor * sin(A + B) * cos(A + B)

    expr = expr.replace(pattern, lambda **kwargs: log_and_replace(pattern, 0))

    if print_replacements and replacements_log:
        print("Fast oscillation approximations applied to:")
        for r in replacements_log:
            print("  ", r)

    return expr


# Update the integral computation function to include simplification inside the parallelized task
def compute_integral_pair_simplified(
    args: Tuple[sp.Expr, sp.Expr, sp.Symbol, sp.Symbol, sp.Symbol, sp.Symbol],
) -> sp.Expr:
    df1, df2, t, T, R, sigma_m, phi = args
    product = sp.expand(df1 * df2)
    product = sp.simplify(product)
    product = apply_fast_oscillation_approximations(product, t, omega, phi)
    integral = sp.integrate(product, (t, 0, T))
    scaled = integral * R / sigma_m**2
    expr = sp.expand(scaled)  # Expand basic algebra first
    expr = sp.expand_trig(expr)  # Expand trig products (e.g., sin^2)
    expr = sp.factor(expr)  # Pull out common exponentials
    expr = sp.cancel(expr)  # Clean up rational forms
    expr = sp.collect(expr, T)  # Group by powers of T (or lambda_D, etc.)
    # expr = sp.simplify(expr)
    return expr


def compute_fisher_matrix_serial_simplified(
    model: sp.Expr,
    parameters: list[sp.Symbol],
    t: sp.Symbol,
    T: sp.Symbol,
    sigma_m: sp.Symbol,
    N: sp.Symbol,
) -> sp.Matrix:
    """
    Computes the Fisher Information Matrix (FIM) without parallelism,
    using simplified symbolic integrals over all parameter pairs.

    Args:
        model: symbolic expression of the model function
        parameters: list of parameters to differentiate with respect to
        t: time symbol
        T: total acquisition time symbol
        sigma_m: measurement noise
        N: number of samples

    Returns:
        sympy.Matrix: symbolic Fisher Information Matrix
    """
    R = N / T
    n = len(parameters)

    entries: list[sp.Expr] = []

    for p1 in parameters:
        df1 = sp.diff(model, p1)
        for p2 in parameters:
            df2 = sp.diff(model, p2)
            result = compute_integral_pair_simplified((df1, df2, t, T, R, sigma_m, phi))
            entries.append(result)

    return sp.Matrix(n, n, entries)


# this was an attempt to parallelize inverse but its not really needed
def compute_variance_schur(args):
    i, FIM = args
    D = FIM[i, i]
    if i == 0:
        return i, sp.cancel(1 / D)
    A = FIM[:i, :i]
    B = FIM[:i, i]
    C = FIM[i, :i]
    try:
        A_inv = A.inv()
    except Exception:
        return i, sp.nan  # Or log and skip
    schur = D - C @ A_inv @ B
    return i, sp.cancel(1 / schur)


# Symbols
t = sp.symbols("t", positive=True, real=True)
T = sp.symbols("T", positive=True, real=True, nonzero=True)
A = sp.symbols("A", positive=True, real=True, nonzero=True)
N = sp.symbols("N", positive=True, real=True, nonzero=True)
sigma_m = sp.symbols("sigma_m", positive=True, real=True, nonzero=True)
lambda_D = sp.symbols("lambda_D", real=True, positive=True, nonzero=True)
omega = sp.symbols("omega", real=True, positive=True, nonzero=True)
phi = sp.symbols("phi", real=True)

# Model
f = A * sp.exp(-lambda_D * t) * sp.sin(omega * t + phi)

# Parameter list
params = [A, lambda_D, omega, phi]
param_indices = {param: i for i, param in enumerate(params)}

exp(-2*lambda_D*t)*sin(omega*t + phi)**2

# integrands

In [ ]:
p1 = params[0]
p2 = params[0]
df1 = sp.diff(f, p1)
df2 = sp.diff(f, p2)
product = sp.expand(df1 * df2)
product

In [206]:
# inspect the integerands

for ii in range(4):
    for jj in range(4):
        print(f"FIM[{ii}, {jj}]:")
        df1 = sp.diff(f, params[ii])
        df2 = sp.diff(f, params[jj])
        product = sp.expand(df1 * df2)
        print(product)
        product = apply_fast_oscillation_approximations(product, t, omega, phi)
        print(product)
        print("\n")

FIM[0, 0]:
exp(-2*lambda_D*t)*sin(omega*t + phi)**2
Fast oscillation approximations applied to:
   sin(omega*t + phi_*sign_)**2
exp(-2*lambda_D*t)/2


FIM[0, 1]:
-A*t*exp(-2*lambda_D*t)*sin(omega*t + phi)**2
Fast oscillation approximations applied to:
   sin(omega*t + phi_*sign_)**2
-A*t*exp(-2*lambda_D*t)/2


FIM[0, 2]:
A*t*exp(-2*lambda_D*t)*sin(omega*t + phi)*cos(omega*t + phi)
Fast oscillation approximations applied to:
   prefactor_*sin(A_ + B_)*cos(A_ + B_)
   prefactor_*sin(A_ + B_)*cos(A_ + B_)
0


FIM[0, 3]:
A*exp(-2*lambda_D*t)*sin(omega*t + phi)*cos(omega*t + phi)
Fast oscillation approximations applied to:
   prefactor_*sin(A_ + B_)*cos(A_ + B_)
   prefactor_*sin(A_ + B_)*cos(A_ + B_)
0


FIM[1, 0]:
-A*t*exp(-2*lambda_D*t)*sin(omega*t + phi)**2
Fast oscillation approximations applied to:
   sin(omega*t + phi_*sign_)**2
-A*t*exp(-2*lambda_D*t)/2


FIM[1, 1]:
A**2*t**2*exp(-2*lambda_D*t)*sin(omega*t + phi)**2
Fast oscillation approximations applied to:
   sin(omega*t + phi_*s

In [ ]:
ii = 0
jj = 2
print(f"FIM[{ii}, {jj}]:")
df1 = sp.diff(f, params[ii])
df2 = sp.diff(f, params[jj])
product = sp.expand(df1 * df2)
print(product)
product = apply_fast_oscillation_approximations(product, t, omega, phi)
print(product)
print("\n")

parallel compute matrix


In [150]:
FIM[0, 0]

(2*N*lambda_D**2*exp(2*T*lambda_D)*sin(phi)**2 - 2*N*lambda_D**2*sin(phi)**2*cos(T*omega)**2 - 4*N*lambda_D**2*sin(phi)*sin(T*omega)*cos(phi)*cos(T*omega) - 2*N*lambda_D**2*sin(T*omega)**2*cos(phi)**2 + 2*N*lambda_D*omega*exp(2*T*lambda_D)*sin(phi)*cos(phi) + 2*N*lambda_D*omega*sin(phi)**2*sin(T*omega)*cos(T*omega) + 2*N*lambda_D*omega*sin(phi)*sin(T*omega)**2*cos(phi) - 2*N*lambda_D*omega*sin(phi)*cos(phi)*cos(T*omega)**2 - 2*N*lambda_D*omega*sin(T*omega)*cos(phi)**2*cos(T*omega) + N*omega**2*exp(2*T*lambda_D)*sin(phi)**2 + N*omega**2*exp(2*T*lambda_D)*cos(phi)**2 - N*omega**2*sin(phi)**2*sin(T*omega)**2 - N*omega**2*sin(phi)**2*cos(T*omega)**2 - N*omega**2*sin(T*omega)**2*cos(phi)**2 - N*omega**2*cos(phi)**2*cos(T*omega)**2)/(T*(4*lambda_D**3*sigma_m**2*exp(2*T*lambda_D) + 4*lambda_D*omega**2*sigma_m**2*exp(2*T*lambda_D)))

In [3]:
FIM

Matrix([
[                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

# compute the FIM

In [208]:
# runtime ~30 s
print("parallel compute matrix")
FIM = compute_fisher_matrix_serial_simplified(f, params, t, T, sigma_m, N)
# Parallel Schur complement variance estimation

parallel compute matrix
Fast oscillation approximations applied to:
   sin(omega*t + phi*sign_)**2
Fast oscillation approximations applied to:
   sin(omega*t + phi*sign_)**2
Fast oscillation approximations applied to:
   prefactor_*sin(A_ + B_)*cos(A_ + B_)
Fast oscillation approximations applied to:
   prefactor_*sin(A_ + B_)*cos(A_ + B_)
Fast oscillation approximations applied to:
   sin(omega*t + phi*sign_)**2
Fast oscillation approximations applied to:
   sin(omega*t + phi*sign_)**2
Fast oscillation approximations applied to:
   prefactor_*sin(A_ + B_)*cos(A_ + B_)
Fast oscillation approximations applied to:
   prefactor_*sin(A_ + B_)*cos(A_ + B_)
Fast oscillation approximations applied to:
   prefactor_*sin(A_ + B_)*cos(A_ + B_)
Fast oscillation approximations applied to:
   prefactor_*sin(A_ + B_)*cos(A_ + B_)
Fast oscillation approximations applied to:
   sin(omega*t + phi*sign_)**2
Fast oscillation approximations applied to:
   sin(omega*t + phi*sign_)**2
Fast oscillation appro

In [209]:
FIM

Matrix([
[                          (N*exp(2*T*lambda_D) - N)*exp(-2*T*lambda_D)/(4*T*lambda_D*sigma_m**2),                                       (2*A*N*T*lambda_D - A*N*exp(2*T*lambda_D) + A*N)*exp(-2*T*lambda_D)/(8*T*lambda_D**2*sigma_m**2),                                                                                                                                      0,                                                                                                          0],
[(2*A*N*T*lambda_D - A*N*exp(2*T*lambda_D) + A*N)*exp(-2*T*lambda_D)/(8*T*lambda_D**2*sigma_m**2), (-2*A**2*N*T**2*lambda_D**2 - 2*A**2*N*T*lambda_D + A**2*N*exp(2*T*lambda_D) - A**2*N)*exp(-2*T*lambda_D)/(8*T*lambda_D**3*sigma_m**2),                                                                                                                                      0,                                                                                                          0],
[                              

# inverse

In [210]:
import importlib
import sym_functions

importlib.reload(sym_functions)

<module 'sym_functions' from '/workspaces/repo/src/sym_functions.py'>

In [211]:
# runtime ~ 30s
FIM_inv = sym_functions.inverse_via_symmetric_substitution(FIM)

creating template for symbolic symmetric inverse
applying substitution to symbolic inverse


In [212]:
FIM_inv

Matrix([
[       (-(A**2*N*exp(2*T*lambda_D) - A**2*N)*(-2*A**2*N*T**2*lambda_D**2 - 2*A**2*N*T*lambda_D + A**2*N*exp(2*T*lambda_D) - A**2*N)**2*exp(-6*T*lambda_D)/(256*T**3*lambda_D**7*sigma_m**6) + (-2*A**2*N*T*lambda_D + A**2*N*exp(2*T*lambda_D) - A**2*N)**2*(-2*A**2*N*T**2*lambda_D**2 - 2*A**2*N*T*lambda_D + A**2*N*exp(2*T*lambda_D) - A**2*N)*exp(-6*T*lambda_D)/(512*T**3*lambda_D**7*sigma_m**6))/(-(N*exp(2*T*lambda_D) - N)*(A**2*N*exp(2*T*lambda_D) - A**2*N)*(-2*A**2*N*T**2*lambda_D**2 - 2*A**2*N*T*lambda_D + A**2*N*exp(2*T*lambda_D) - A**2*N)**2*exp(-8*T*lambda_D)/(1024*T**4*lambda_D**8*sigma_m**8) + (N*exp(2*T*lambda_D) - N)*(-2*A**2*N*T*lambda_D + A**2*N*exp(2*T*lambda_D) - A**2*N)**2*(-2*A**2*N*T**2*lambda_D**2 - 2*A**2*N*T*lambda_D + A**2*N*exp(2*T*lambda_D) - A**2*N)*exp(-8*T*lambda_D)/(2048*T**4*lambda_D**8*sigma_m**8) + (A**2*N*exp(2*T*lambda_D) - A**2*N)*(2*A*N*T*lambda_D - A*N*exp(2*T*lambda_D) + A*N)**2*(-2*A**2*N*T**2*lambda_D**2 - 2*A**2*N*T*lambda_D + A**2*N*exp(2*T*l

In [213]:
index_a = param_indices[A]
var_A = FIM_inv[index_a, index_a]
index_phi = param_indices[phi]
var_phi = FIM_inv[index_phi, index_phi]
index_omega = param_indices[omega]
var_omega = FIM_inv[index_omega, index_omega]
index_lambda = param_indices[lambda_D]
var_lambda = FIM_inv[index_lambda, index_lambda]

In [214]:
sym_functions.sym_info(var_lambda)

symbolic expression has 295 terms and length 1_188
inbuilt count:
  sp.pow                  64
  sp.trig                  0
symbols count:            10
  N                       45
  T                       41
  lambda_D                41
  A                       37
  sigma_m                  6
  1/2048                   2
  -1/128                   1
  -1/1024                  1
  1/256                    1
  -1/4096                  1


In [215]:
FIM_inv_symp = FIM_inv
FIM_inv_symp = FIM_inv_symp.applyfunc(sp.expand)
FIM_inv_symp = FIM_inv_symp.applyfunc(sp.expand_trig)
FIM_inv_symp = FIM_inv_symp.applyfunc(sp.factor)
FIM_inv_symp = FIM_inv_symp.applyfunc(sp.cancel)
FIM_inv_symp = FIM_inv_symp.applyfunc(sp.simplify)

In [218]:
index_a = param_indices[A]
var_A = FIM_inv_symp[index_a, index_a]
index_phi = param_indices[phi]
var_phi = FIM_inv_symp[index_phi, index_phi]
index_omega = param_indices[omega]
var_omega = FIM_inv_symp[index_omega, index_omega]
index_lambda = param_indices[lambda_D]
var_lambda = FIM_inv_symp[index_lambda, index_lambda]

In [220]:
var_A

8*T*lambda_D*sigma_m**2*(2*T**2*lambda_D**2 + 2*T*lambda_D - exp(2*T*lambda_D) + 1)*exp(2*T*lambda_D)/(N*(4*T**2*lambda_D**2*exp(2*T*lambda_D) - exp(4*T*lambda_D) + 2*exp(2*T*lambda_D) - 1))

In [219]:
sym_functions.sym_info(var_lambda)

symbolic expression has 36 terms and length 163
inbuilt count:
  sp.pow                   7
  sp.trig                  0
symbols count:             5
  T                        7
  lambda_D                 7
  N                        1
  A                        1
  sigma_m                  1


In [243]:
var_omega

16*T*lambda_D**3*sigma_m**2*(1 - exp(2*T*lambda_D))*exp(2*T*lambda_D)/(A**2*N*(4*T**2*lambda_D**2*exp(2*T*lambda_D) - exp(4*T*lambda_D) + 2*exp(2*T*lambda_D) - 1))

In [240]:
# generate code

for name, expr in zip(
    ["var_A", "var_phi", "var_omega", "var_lambda"],
    [var_A, var_phi, var_omega, var_lambda],
):
    print(f"{name}")
    # Map symbols to variable names in function signature
    rename_map = {
        A: "amp",
        N: "samp_num",
        T: "samp_time",
        sigma_m: "sigma_obs",
        lambda_D: "damp_rate",
        omega: "omega",  # leave unchanged
        phi: "phi",  # leave unchanged
        t: "t",  # used internally, not in function signature
    }

    # Convert to substituted code string

    subs = {k: sp.Symbol(v) for k, v in rename_map.items()}
    expr_sub = expr.subs(subs)
    pycode_expr = sp.printing.pycode(expr_sub)
    print(pycode_expr)
    print("\n")

var_A
8*damp_rate*samp_time*sigma_obs**2*(2*damp_rate**2*samp_time**2 + 2*damp_rate*samp_time - math.exp(2*damp_rate*samp_time) + 1)*math.exp(2*damp_rate*samp_time)/(samp_num*(4*damp_rate**2*samp_time**2*math.exp(2*damp_rate*samp_time) - math.exp(4*damp_rate*samp_time) + 2*math.exp(2*damp_rate*samp_time) - 1))


var_phi
8*damp_rate*samp_time*sigma_obs**2*(2*damp_rate**2*samp_time**2 + 2*damp_rate*samp_time - math.exp(2*damp_rate*samp_time) + 1)*math.exp(2*damp_rate*samp_time)/(amp**2*samp_num*(4*damp_rate**2*samp_time**2*math.exp(2*damp_rate*samp_time) - math.exp(4*damp_rate*samp_time) + 2*math.exp(2*damp_rate*samp_time) - 1))


var_omega
16*damp_rate**3*samp_time*sigma_obs**2*(1 - math.exp(2*damp_rate*samp_time))*math.exp(2*damp_rate*samp_time)/(amp**2*samp_num*(4*damp_rate**2*samp_time**2*math.exp(2*damp_rate*samp_time) - math.exp(4*damp_rate*samp_time) + 2*math.exp(2*damp_rate*samp_time) - 1))


var_lambda
16*damp_rate**3*samp_time*sigma_obs**2*(1 - math.exp(2*damp_rate*samp_time))*

# truncated inverse
ignore non diagonal terms

In [143]:
from typing import List, Tuple


def zero_except(mat_in: sp.Matrix, keep_pairs: List[Tuple[int, int]]) -> sp.Matrix:
    """
    Returns a copy of the matrix with all off-diagonal elements zeroed out,
    except those in the keep_pairs list (applied symmetrically).

    Args:
        F: SymPy square matrix
        keep_pairs: list of (i, j) index pairs to preserve (both [i,j] and [j,i])

    Returns:
        A new matrix with only specified off-diagonals (and the diagonal) preserved.
    """
    n = mat_in.shape[0]
    assert mat_in.shape == (n, n), "Matrix must be square"

    # Build symmetric set of allowed pairs
    keep = {(i, j) for (i, j) in keep_pairs}
    keep |= {(j, i) for (i, j) in keep_pairs}

    F_new = mat_in.copy()
    for i in range(n):
        for j in range(n):
            if i == j:
                continue  # Always keep diagonal
            if (i, j) not in keep:
                F_new[i, j] = 0
    return F_new


FIM_simplified = FIM.copy()
allowed_pairs = []
index_pairs = [(param_indices[a], param_indices[b]) for a, b in allowed_pairs]
FIM_simplified = zero_except(FIM_simplified, index_pairs)
FIM_simplified

Matrix([
[(2*N*lambda_D**2*exp(2*T*lambda_D)*sin(phi)**2 - 2*N*lambda_D**2*sin(phi)**2*cos(T*omega)**2 - 4*N*lambda_D**2*sin(phi)*sin(T*omega)*cos(phi)*cos(T*omega) - 2*N*lambda_D**2*sin(T*omega)**2*cos(phi)**2 + 2*N*lambda_D*omega*exp(2*T*lambda_D)*sin(phi)*cos(phi) + 2*N*lambda_D*omega*sin(phi)**2*sin(T*omega)*cos(T*omega) + 2*N*lambda_D*omega*sin(phi)*sin(T*omega)**2*cos(phi) - 2*N*lambda_D*omega*sin(phi)*cos(phi)*cos(T*omega)**2 - 2*N*lambda_D*omega*sin(T*omega)*cos(phi)**2*cos(T*omega) + N*omega**2*exp(2*T*lambda_D)*sin(phi)**2 + N*omega**2*exp(2*T*lambda_D)*cos(phi)**2 - N*omega**2*sin(phi)**2*sin(T*omega)**2 - N*omega**2*sin(phi)**2*cos(T*omega)**2 - N*omega**2*sin(T*omega)**2*cos(phi)**2 - N*omega**2*cos(phi)**2*cos(T*omega)**2)/(T*(4*lambda_D**3*sigma_m**2*exp(2*T*lambda_D) + 4*lambda_D*omega**2*sigma_m**2*exp(2*T*lambda_D))),                                                                                                                                                         

In [144]:
# runtime ~ 30s
FIM_simplified_inv = sym_functions.inverse_via_symmetric_substitution(FIM_simplified)

inverting


In [146]:
var_A = FIM_simplified_inv[index_a, index_a]
sym_functions.sym_info(var_A)

symbolic expression has 188 terms and length 834
inbuilt count:
  sp.pow                  34
  sp.trig                 17
symbols count:             6
  omega                   26
  T                       21
  phi                     19
  lambda_D                17
  N                       15
  sigma_m                  2


In [149]:
std_A = sp.sqrt(var_A).simplify()
std_A

2*sqrt(T)*sqrt(lambda_D)*sigma_m*sqrt(-1/(lambda_D**2*exp(2*T*lambda_D)*cos(2*phi) - lambda_D**2*exp(2*T*lambda_D) - lambda_D**2*cos(2*T*omega + 2*phi) + lambda_D**2 - lambda_D*omega*exp(2*T*lambda_D)*sin(2*phi) + lambda_D*omega*sin(2*T*omega + 2*phi) - omega**2*exp(2*T*lambda_D) + omega**2))*sqrt(lambda_D**2 + omega**2)*exp(T*lambda_D)/sqrt(N)

# Common expressions

In [110]:
replacements, reduced = sp.cse(a)
assert len(reduced) == 1, "Expected a single reduced expression"
reduced = reduced[0]
sym_info(reduced)

symbolic expression has 60 terms and length 330
inbuilt count:
  sp.pow                   3
  sp.trig                  0
symbols count:            33
  x265                     3
  x266                     3
  x148                     2
  x269                     2
  x137                     2
  x260                     2
  x64                      2
  x268                     2
  x97                      2
  x252                     2
  x156                     2
  x251                     2
  x248                     2
  x96                      2
  x259                     2
  x270                     2
  x144                     2
  x271                     2
  x147                     2
  x162                     2
  x77                      2
  x250                     2
  x261                     2
  x249                     1
  x155                     1
  x149                     1
  x76                      1
  x78                      1
  x138                     1
  x143   

In [114]:
simp_reduced = sp.expand(reduced)
simp_reduced = sp.factor(simp_reduced)
simp_reduced = sp.cancel(simp_reduced)
simp_reduced = sp.factor(simp_reduced)
simp_reduced = sp.collect(simp_reduced, "x143")
sym_info(simp_reduced)
simp_reduced

symbolic expression has 94 terms and length 453
inbuilt count:
  sp.pow                  20
  sp.trig                  0
symbols count:            33
  x76                     17
  x265                     3
  x266                     3
  x148                     2
  x269                     2
  x137                     2
  x64                      2
  x268                     2
  x97                      2
  x252                     2
  x156                     2
  x251                     2
  x248                     2
  x96                      2
  x259                     2
  x261                     2
  x270                     2
  x144                     2
  x143                     2
  x271                     2
  x147                     2
  x162                     2
  x77                      2
  x250                     2
  x260                     2
  x249                     1
  x155                     1
  x149                     1
  x78                      1
  x138   

x143**4*x76**4*(x148 - x156 + x162*x248 - x162*x250 + x97)/(-x138*x261*x76**4 + x143**4*(x137*x144*x260*x268*x76**4 - x137*x147*x251*x270*x76**4 + x144*x149*x269*x76**4*x77 + x145*x261*x266*x76**4*x96 + x147*x252*x266*x76**4*x78 + x148*x265*x76**4 - 2*x155*x266*x269*x76**4 - x156*x265*x76**4 - x247*x259*x76**4 + x248*x271*x76**4 + x249*x259*x76**4 - x250*x271*x76**4 + x251*x268*x64*x76**4*x77 - x252*x65 - x260*x270*x64*x76**4*x96 + x265*x76**4*x97))

In [ ]:
simp_reduced

In [ ]:
import importlib
import sym_functions

importlib.reload(sym_functions)

<module 'sym_functions' from '/workspaces/repo/src/sym_functions.py'>

In [45]:
a.count(sp.Pow)

7606

In [43]:
a.atoms()

{-1,
 -10,
 -12,
 -16,
 -2,
 -3,
 -4,
 -6,
 -8,
 12,
 16,
 2,
 24,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 N,
 T,
 lambda_D,
 omega,
 phi,
 sigma_m}

In [39]:
a = FIM_inv[1, 1]
sym_info(a)
print("cancel")
a = sp.cancel(a)
sym_info(a)
print("factor")
a = sp.factor(a)
sym_info(a)
print("cancel")
a = sp.cancel(a)
sym_info(a)
a

symbolic expression has 36_575 terms and length 160_112
cancel


KeyboardInterrupt: 

In [ ]:
FIM_inv[1, 1].simplify()

In [40]:
FIM = sp.expand(FIM)  # Expand basic algebra first

In [ ]:
# Invert and simplify
FIM_inv = FIM.inv()

In [ ]:
FIM_inv = FIM_inv.applyfunc(sp.cancel)  # Clean up each entry

In [ ]:
#
# print("parallel compute variances")
# with Pool(cpu_count()) as pool:
#     results = pool.map(compute_variance_schur, [(i, FIM) for i in range(FIM.shape[0])])

parallel compute variances


Process ForkPoolWorker-205:
Process ForkPoolWorker-210:


KeyboardInterrupt: 

In [ ]:
def derive_covariance(
    model: sp.Expr,
    parameters: list[sp.Symbol],
    t: sp.Symbol,
    T: sp.Symbol,
    sigma_m: sp.Symbol,
    N: sp.Symbol,
) -> Tuple[Dict[str, sp.Expr], sp.Matrix]:
    """Compute estimator variances and full covariance matrix via Fisher analysis.

    Parameters
    ----------
    model : Expr
        The symbolic model function f(t).
    parameters : list of Symbol
        Parameter symbols to estimate.
    t : Symbol
        The integration variable (e.g., time).
    T : Symbol
        Total observation time.
    sigma_m : Symbol
        Standard deviation of measurement noise.
    N : Symbol
        Number of samples.

    Returns
    -------
    Tuple[Dict[str, Expr], Matrix]
        A dictionary mapping each parameter to its variance, and
        the full symbolic covariance matrix (inverse FIM).
    """
    R = N / T
    FIM = sp.Matrix(
        [
            [
                sp.integrate(sp.diff(model, p1) * sp.diff(model, p2), (t, 0, T))
                * R
                / sigma_m**2
                for p2 in parameters
            ]
            for p1 in parameters
        ]
    )
    covariance = FIM.inv()
    variances = {
        str(param): sp.simplify(covariance[i, i]) for i, param in enumerate(parameters)
    }
    return variances, covariance


# Symbols
t, T = sp.symbols("t T", positive=True, real=True)
A, sigma_m, N = sp.symbols("A sigma_m N", positive=True)
lambda_D, omega, phi = sp.symbols("lambda_D omega phi", real=True)

# Model
f = A * sp.exp(-lambda_D * t) * sp.sin(omega * t + phi)

# Parameter list
params = [A, lambda_D, omega, phi]

# Compute
variances, covariance_matrix = derive_covariance(f, params, t, T, sigma_m, N)

# Display
print("Variances:")
for name, var in variances.items():
    print(f"{name}: {var}\n")

In [ ]:
def compare_with_target(expressions: Dict[str, sp.Expr]) -> Dict[str, bool]:
    """Compare derived variances with the target formulas from the derivation.

    Parameters
    ----------
    expressions : Dict[str, sp.Expr]
        Dictionary containing the derived variances ``var_A``, ``var_phi``
        and ``var_omega``.

    Returns
    -------
    Dict[str, bool]
        Dictionary indicating whether each derived expression is
        algebraically equivalent to the corresponding target expression.
    """

    lambda_D, T, A, sigma_m, N = sp.symbols("lambda_D T A sigma_m N", positive=True)
    # Target expressions from the supplied derivation (see Eqs. (A.36)–(A.38)).
    var_A_target: sp.Expr = sp.simplify(
        (2 * lambda_D * sigma_m**2 * T * (sp.coth(lambda_D * T) + 1)) / N
    )
    var_phi_target: sp.Expr = sp.simplify(
        (
            4
            * lambda_D
            * sigma_m**2
            * T
            * (-2 * lambda_D * T * (lambda_D * T + 1) + sp.exp(2 * lambda_D * T) - 1)
        )
        / (A**2 * N * (-2 * lambda_D**2 * T**2 + sp.cosh(2 * lambda_D * T) - 1))
    )
    var_omega_target: sp.Expr = sp.simplify(
        (8 * lambda_D**3 * sigma_m**2 * T * (sp.exp(2 * lambda_D * T) - 1))
        / (A**2 * N * (-2 * lambda_D**2 * T**2 + sp.cosh(2 * lambda_D * T) - 1))
    )

    # Substitution dictionary for matching symbols in derived expressions
    subs_dict = {
        sp.symbols("lambda_D", positive=True): lambda_D,
        sp.symbols("T", positive=True): T,
        sp.symbols("A", positive=True): A,
        sp.symbols("sigma_m", positive=True): sigma_m,
        sp.symbols("N", positive=True): N,
    }

    # Compare via ratio: two expressions are equivalent if their ratio simplifies to 1
    def match(expr1: sp.Expr, expr2: sp.Expr) -> bool:
        """Return True if two expressions are equivalent.

        The comparison first attempts symbolic simplification of the ratio
        ``expr1/expr2`` to 1.  If that fails (e.g. due to latent
        exponential–hyperbolic identities), a numerical check is
        performed at several positive values of ``lambda_D`` and ``T``.
        """
        ratio = sp.simplify(expr1 / expr2)
        # Direct symbolic check
        if sp.simplify(ratio - 1) == 0:
            return True
        # Numerical sampling fallback
        numeric_tests = [(0.1, 2.0), (0.5, 1.2), (0.2, 3.5)]
        lambda_D_sym, T_sym = sp.symbols("lambda_D T", positive=True)
        f_lambda = lambda_D_sym
        f_T = T_sym
        # Create callable function for ratio using lambdify to ensure numeric evaluation
        ratio_func = sp.lambdify((lambda_D_sym, T_sym), ratio, "numpy")
        import math

        for lam_val, T_val in numeric_tests:
            try:
                val = ratio_func(lam_val, T_val)
            except Exception:
                return False
            if not math.isfinite(val) or abs(val - 1.0) > 1e-10:
                return False
        return True

    var_A_match: bool = match(expressions["var_A"].subs(subs_dict), var_A_target)
    var_phi_match: bool = match(expressions["var_phi"].subs(subs_dict), var_phi_target)
    var_omega_match: bool = match(
        expressions["var_omega"].subs(subs_dict), var_omega_target
    )

    return {
        "var_A": var_A_match,
        "var_phi": var_phi_match,
        "var_omega": var_omega_match,
    }